<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/TineTunning_Testing_For_AviationQADataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/Cloud_curious/blob/master/FineTuning_Mistral_7b_hfdeployment_dataset_AviationQA.ipynb

In [1]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is i

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [4]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
#model_name = "frankmorales2020/Mistral-7B-Instruct-v0.1_AviationQA" # 07/06/2024

model_name = 'mistralai/Mistral-7B-Instruct-v0.3'

# set device
device = 'cuda'

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
#del model

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024, 08 MARCH 2024 and 27 MAY 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    #attn_implementation="flash_attention_2", ## BertLMHeadModel does not support Flash Attention 2.0 yet
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [8]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "frankmorales2020/Mistral-7B-Instruct-v0.1_AviationQA" # 07/06/2024
#peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2"
peft_model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

# Load Model with PEFT adapter
#model = AutoPeftModelForCausalLM.from_pretrained(

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The model 'MistralForCausalLM' is not supported for document-question-answering. Supported models are ['LayoutLMForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering'].


In [9]:
# Convert dataset to OAI messages
system_message = """You are an AVIATION EXPERT. Users will ask you questions in English and you will generate ANSWERS based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message},
      {"role": "user", "content": sample["Question"]},
      {"role": "assistant", "content": sample["Answer"]}
    ]
  }


In [10]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")

dataset = load_dataset("sakharamg/AviationQA")
#dataset = dataset.map(create_prompt_formats)

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation)


# save datasets to disk
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

Preprocessing dataset AviationQA


Generating train split:   0%|          | 0/1057986 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11888 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10807 [00:00<?, ? examples/s]

Map:   0%|          | 0/1057986 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/10807 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1058 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

5463726

In [13]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [14]:
dataset

Dataset({
    features: ['id', 'Question', 'Answer', 'messages'],
    num_rows: 10807
})

In [15]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]
dataset_final_Messages=dataset['messages'][0:nrec]

In [16]:
dataset_final_Messages[1]

[{'content': 'You are an AVIATION EXPERT. Users will ask you questions in English and you will generate ANSWERS based on the provided SCHEMA.\nSCHEMA:\n{schema}',
  'role': 'system'},
 {'content': 'When was the Last Flight Review or Equivalent done of the pilot for the accident no. GAA17CA495?',
  'role': 'user'},
 {'content': 'May 13, 2017', 'role': 'assistant'}]

In [17]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [18]:
datasetF

,id,Question,Answer
0,a4905077116,What is the name of the Engine Manufacturer of...,Lycoming
1,a7510929859,When was the Last Flight Review or Equivalent ...,"May 13, 2017"
2,a5430892651,What was the flight conducted under for the ac...,Part 91: General aviation - Personal
3,a9405153070,What was the Observation Time of the accident ...,13:54 Local
4,a8193000066,What is the name of the Engine Manufacturer of...,LYCOMING
5,a9610719319,What is the Airworthiness Certificate of the a...,Normal
6,a7001234003,Was there any Operating Certificate(s) held of...,None
7,a9135502144,How many Seats are there in the aircraft beari...,2
8,a157434574,What is the Serial Number of the aircraft bear...,07-12-03
9,a1523521076,What is the Airframe Total Time of the acciden...,10663 Hrs as of last inspection


In [19]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [20]:
eval_dataset

Dataset({
    features: ['id', 'Question', 'Answer', 'messages'],
    num_rows: 10807
})

In [21]:
eval_dataset[0]["messages"][:2]

[{'content': 'You are an AVIATION EXPERT. Users will ask you questions in English and you will generate ANSWERS based on the provided SCHEMA.\nSCHEMA:\n{schema}',
  'role': 'system'},
 {'content': 'What is the name of the Engine Manufacturer of the accident no. WPR15CA128?',
  'role': 'user'}]

In [22]:
from transformers import pipeline

#qa_pipeline = pipeline("question-answering")
#qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

#generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [23]:
eval_dataset[2]['Question']

'What was the flight conducted under for the accident no. GAA18CA423?'

In [24]:
eval_dataset[2]['Answer']

'Part 91: General aviation - Personal'

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [105]:
from random import randint
rand_idx = randint(0, len(eval_dataset))
#rand_idx = 3297

In [106]:
#rand_idx = randint(0, len(eval_dataset))
modified_conversation = [
    {"role": "user", "content": eval_dataset[rand_idx]["messages"][:3][1]['content']},
    {"role": "assistant", "content": eval_dataset[rand_idx]["messages"][:3][2]['content']}
]

In [108]:
rand_idx

3564

In [109]:
modified_conversation

[{'role': 'user',
  'content': 'What did the pilot report on the accident no. WPR13CA155?'},
 {'role': 'assistant',
  'content': 'During descent for an off-airport pinnacle landing, the helicopter encountered a downdraft about 75 to 100 feet above ground level (agl), and the pilot receiving instruction increased collective input. The flight instructor advised the pilot receiving instruction to be less aggressive using the collective as the flight continued to the landing area. As the helicopter descended through about 10 to 25 feet agl, it encountered a second downdraft, and the pilot receiving instruction increased collective again, which was followed by a decrease in rotor rpm. The flight instructor took control of the helicopter as it contacted the ground and then became airborne again. The helicopter yawed to the right and landed upright on sloping terrain. Postaccident examination of the helicopter revealed that the tailboom was substantially damaged just aft of the fuselage attac

In [110]:
#model.generation_config.pad_token_id = tokenizer.pad_token_id

# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])

# Modify the conversation to ensure alternating roles if needed
# For example, if the first two messages are both user messages:
modified_conversation = [
    {"role": "user", "content": eval_dataset[rand_idx]["messages"][:3][1]['content']},
    {"role": "assistant", "content": eval_dataset[rand_idx]["messages"][:3][2]['content']}
]

#print(modified_conversation)

#Use the chat template to format the input for the language model
prompt = pipe.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)


# Replace 'text-generation' with the appropriate task for your model
# For example, if you're using a causal language model:
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Pass the prompt to the generation pipeline
outputs = generation_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.9,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [113]:
outputs

[{'generated_text': "<s>[INST] What did the pilot report on the accident no. WPR13CA155? [/INST]During descent for an off-airport pinnacle landing, the helicopter encountered a downdraft about 75 to 100 feet above ground level (agl), and the pilot receiving instruction increased collective input. The flight instructor advised the pilot receiving instruction to be less aggressive using the collective as the flight continued to the landing area. As the helicopter descended through about 10 to 25 feet agl, it encountered a second downdraft, and the pilot receiving instruction increased collective again, which was followed by a decrease in rotor rpm. The flight instructor took control of the helicopter as it contacted the ground and then became airborne again. The helicopter yawed to the right and landed upright on sloping terrain. Postaccident examination of the helicopter revealed that the tailboom was substantially damaged just aft of the fuselage attach point. The pilots reported no pr

In [112]:
#rand_idx = randint(0, len(eval_dataset))
print(f"Question:\n{eval_dataset[rand_idx]['Question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['Answer']}")
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Question:
What did the pilot report on the accident no. WPR13CA155?

Original Answer:
During descent for an off-airport pinnacle landing, the helicopter encountered a downdraft about 75 to 100 feet above ground level (agl), and the pilot receiving instruction increased collective input. The flight instructor advised the pilot receiving instruction to be less aggressive using the collective as the flight continued to the landing area. As the helicopter descended through about 10 to 25 feet agl, it encountered a second downdraft, and the pilot receiving instruction increased collective again, which was followed by a decrease in rotor rpm. The flight instructor took control of the helicopter as it contacted the ground and then became airborne again. The helicopter yawed to the right and landed upright on sloping terrain. Postaccident examination of the helicopter revealed that the tailboom was substantially damaged just aft of the fuselage attach point. The pilots reported no preimpact me